# Лабораторная работа №3: Проведение исследований с решающим деревом

## Задача классификации

## 2. Создание бейзлайна и оценка качества

In [ ]:
X = bl_cdata.drop('HeartDisease', axis=1) 
y = bl_cdata['HeartDisease'] 


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


model = DecisionTreeClassifier(random_state=42)


model.fit(X_train_scaled, y_train)


y_pred = model.predict(X_test_scaled)


accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)


print(f"Метрики для классификации (Решающее дерево):")
print(f"  Accuracy: {accuracy:.4f}")
print(f"  Precision: {precision:.4f}")
print(f"  Recall: {recall:.4f}")

In [ ]:
  Accuracy: 0.8043
  Precision: 0.8114
  Recall: 0.8043

Точность для встроенных моделей sklearn оказалась очень хорошей. Попробуем улучшить бейзлайн:

## 3. Улучшение бейзлайна

Проведем улучшение бейзлайна на основе гипотезы о подборе гиперпараметров решающего дерева. Подбор оптимальных значений гиперпараметров max_depth, min_samples_split и min_samples_leaf для модели DecisionTreeClassifier позволит улучшить метрики классификации (Accuracy, Precision, Recall) по сравнению с моделью с параметрами по умолчанию.

In [ ]:
X = bl_cdata.drop('HeartDisease', axis=1)
y = bl_cdata['HeartDisease']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


param_grid = {
    'max_depth': [3, 5, 7, 10, None], 
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 5, 10]
}


model = DecisionTreeClassifier(random_state=42)


grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', verbose=2, n_jobs=-1) 


grid_search.fit(X_train_scaled, y_train)


print(f"Лучшие параметры: {grid_search.best_params_}")

In [ ]:
Лучшие параметры: {'max_depth': 7, 'min_samples_leaf': 10, 'min_samples_split': 2}

Обучаем модели с лучшими параметрами

In [ ]:
best_model = grid_search.best_estimator_

# Делаем предсказания на масштабированных тестовых данных
y_pred = best_model.predict(X_test_scaled)

# Вычисляем метрики
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)

# Выводим метрики
print(f"Метрики для классификации (Решающее дерево с подбором параметров):")
print(f"  Accuracy: {accuracy:.4f}")
print(f"  Precision: {precision:.4f}")
print(f"  Recall: {recall:.4f}")

In [ ]:
  Accuracy: 0.8696
  Precision: 0.8712
  Recall: 0.8696

С помощью отбора признаков и регуляризации получилось незначительно улучшить результаты.

## 4. Имплементация алгоритма машинного обучения

Напишем собственную реализацию решающего дерева для классификации

In [ ]:
import numpy as np
from collections import Counter


class Node:
    """Представляет узел дерева решений."""

    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature  # индекс признака, по которому разделяем
        self.threshold = threshold  # порог разделения
        self.left = left  # левый дочерний узел
        self.right = right  # правый дочерний узел
        self.value = value  # значение листа (если узел листовой)


class DecisionTree:
    """Реализация дерева решений для классификации."""

    def __init__(self, min_samples_split=2, max_depth=10):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.root = None

    def _gini(self, y):
        """Вычисляет индекс Джини."""
        if len(y) == 0:
            return 0
        counts = Counter(y)
        impurity = 1
        for label in counts:
            p = counts[label] / len(y)
            impurity -= p ** 2
        return impurity

    def _information_gain(self, y, left_y, right_y):
        """Вычисляет прирост информации."""
        p = len(left_y) / len(y)
        gain = self._gini(y) - p * self._gini(left_y) - (1 - p) * self._gini(right_y)
        return gain

    def _best_split(self, X, y):
        """Находит наилучшее разбиение для набора данных."""
        best_gain = 0
        best_feature = None
        best_threshold = None

        for feature in range(X.shape[1]):
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                left_indices = X[:, feature] <= threshold
                right_indices = X[:, feature] > threshold
                left_y = y[left_indices]
                right_y = y[right_indices]

                if len(left_y) < 1 or len(right_y) < 1:
                    continue
                gain = self._information_gain(y, left_y, right_y)
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_threshold = threshold
        return best_feature, best_threshold

    def _build_tree(self, X, y, depth=0):
        """Рекурсивно строит дерево решений."""

        if len(y) < self.min_samples_split or depth == self.max_depth or self._gini(y) == 0:
            counts = Counter(y)
            value = max(counts, key=counts.get)
            return Node(value=value)

        feature, threshold = self._best_split(X, y)
        if feature is None:
            counts = Counter(y)
            value = max(counts, key=counts.get)
            return Node(value=value)

        left_indices = X[:, feature] <= threshold
        right_indices = X[:, feature] > threshold
        left_X = X[left_indices]
        right_X = X[right_indices]
        left_y = y[left_indices]
        right_y = y[right_indices]

        left_subtree = self._build_tree(left_X, left_y, depth + 1)
        right_subtree = self._build_tree(right_X, right_y, depth + 1)
        return Node(feature, threshold, left_subtree, right_subtree)

    def fit(self, X, y):
        """Обучает дерево решений."""
        self.root = self._build_tree(X, y)

    def _predict_sample(self, x, node):
        """Предсказывает метку класса для одного образца."""
        if node.value is not None:
            return node.value
        if x[node.feature] <= node.threshold:
            return self._predict_sample(x, node.left)
        else:
            return self._predict_sample(x, node.right)

    def predict(self, X):
        """Предсказывает метки классов для набора образцов."""
        predictions = []
        for x in X:
            predictions.append(self._predict_sample(x, self.root))
        return np.array(predictions)

Обучим модели и оценим их качество

In [ ]:
X = bl_cdata.drop('HeartDisease', axis=1)
y = bl_cdata['HeartDisease'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


model = DecisionTree(max_depth=5)  
model.fit(X_train_scaled, y_train)


y_pred = model.predict(X_test_scaled)


accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)

print("Метрики для нашей модели дерева решений:")
print(f"  Accuracy: {accuracy:.4f}")
print(f"  Precision: {precision:.4f}")
print(f"  Recall: {recall:.4f}")

In [ ]:
  Accuracy: 0.8750
  Precision: 0.8772
  Recall: 0.8750

Точность реализованных вручную моделей в даже лучше библиотечных.

## Задача регрессии

## 2. Создание бейзлайна и оценка качества

In [ ]:
X = bl_rdata.drop(target_variable, axis=1)
y = bl_rdata[target_variable]

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Масштабируем признаки (стандартизация)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Создаем модель решающего дерева для регрессии
model = DecisionTreeRegressor(random_state=42)

# Обучаем модель на масштабированных обучающих данных
model.fit(X_train_scaled, y_train)

# Делаем предсказания на масштабированных тестовых данных
y_pred = model.predict(X_test_scaled)

# Вычисляем метрики для регрессии
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Выводим метрики
print(f"Метрики для регрессии (Решающее дерево):")
print(f"  Mean Absolute Error: {mae:.4f}")
print(f"  R-squared: {r2:.4f}")

In [ ]:
  Mean Absolute Error: 0.0990
  R-squared: 0.6030

Точность для встроенной в Sklearn модели решающего дерева получилась приемлемой. Попробуем её улучшить

## 3. Улучшение бейзлайна

Воспользуемся тем же методом, что и в случае с классификацией

In [ ]:
X = bl_rdata.drop(target_variable, axis=1)
y = bl_rdata[target_variable]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

param_grid = {
    'max_depth': [3, 5, 7, 10, None],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 5, 10]
}


model = DecisionTreeRegressor(random_state=42)

grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_absolute_error', verbose=2, n_jobs=-1)

grid_search.fit(X_train_scaled, y_train)

print(f"Лучшие параметры: {grid_search.best_params_}")

best_model = grid_search.best_estimator_

y_pred = best_model.predict(X_test_scaled)


mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Метрики для регрессии (Решающее дерево с подбором параметров):")
print(f"  Mean Absolute Error: {mae:.4f}")
print(f"  R-squared: {r2:.4f}")

In [ ]:
  Mean Absolute Error: 0.0672
  R-squared: 0.6520

Изменения незначительные

## 4. Имплементация алгоритма машинного обучения

Напишем собственную реализацию решающего дерева для регрессии

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_absolute_error, r2_score


# Кастомная реализация DecisionTreeRegressor
class Node:
    """Представляет узел дерева решений."""

    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature  # индекс признака, по которому разделяем
        self.threshold = threshold  # порог разделения
        self.left = left  # левый дочерний узел
        self.right = right  # правый дочерний узел
        self.value = value  # значение листа (если узел листовой)


class DecisionTreeRegressor:
    """Реализация дерева решений для регрессии."""

    def __init__(self, min_samples_split=2, max_depth=10):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.root = None

    def _variance(self, y):
        """Вычисляет дисперсию."""
        if len(y) == 0:
            return 0
        return np.var(y)

    def _mean_squared_error(self, y):
        """Вычисляет среднеквадратичную ошибку"""
        if len(y) == 0:
            return 0
        mean = np.mean(y)
        return np.mean((y - mean) ** 2)

    def _information_gain(self, y, left_y, right_y):
        """Вычисляет прирост информации, используя дисперсию."""
        p = len(left_y) / len(y)
        # gain = self._variance(y) - p * self._variance(left_y) - (1 - p) * self._variance(right_y)
        gain = self._mean_squared_error(y) - p * self._mean_squared_error(left_y) - (1 - p) * self._mean_squared_error(
            right_y)
        return gain

    def _best_split(self, X, y):
        """Находит наилучшее разбиение для набора данных."""
        best_gain = 0
        best_feature = None
        best_threshold = None

        for feature in range(X.shape[1]):
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                left_indices = X[:, feature] <= threshold
                right_indices = X[:, feature] > threshold
                left_y = y[left_indices]
                right_y = y[right_indices]

                if len(left_y) < 1 or len(right_y) < 1:
                    continue
                gain = self._information_gain(y, left_y, right_y)
                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature
                    best_threshold = threshold
        return best_feature, best_threshold

    def _build_tree(self, X, y, depth=0):
        """Рекурсивно строит дерево решений."""

        if len(y) < self.min_samples_split or depth == self.max_depth or self._mean_squared_error(y) == 0:
            value = np.mean(y)
            return Node(value=value)

        feature, threshold = self._best_split(X, y)
        if feature is None:
            value = np.mean(y)
            return Node(value=value)

        left_indices = X[:, feature] <= threshold
        right_indices = X[:, feature] > threshold
        left_X = X[left_indices]
        right_X = X[right_indices]
        left_y = y[left_indices]
        right_y = y[right_indices]

        left_subtree = self._build_tree(left_X, left_y, depth + 1)
        right_subtree = self._build_tree(right_X, right_y, depth + 1)
        return Node(feature, threshold, left_subtree, right_subtree)

    def fit(self, X, y):
        """Обучает дерево решений."""
        self.root = self._build_tree(X, y)

    def _predict_sample(self, x, node):
        """Предсказывает значение для одного образца."""
        if node.value is not None:
            return node.value
        if x[node.feature] <= node.threshold:
            return self._predict_sample(x, node.left)
        else:
            return self._predict_sample(x, node.right)

    def predict(self, X):
        """Предсказывает значения для набора образцов."""
        predictions = []
        for x in X:
            predictions.append(self._predict_sample(x, self.root))
        return np.array(predictions)


Обучаем имплементированные модели и оцением их качество
     

In [ ]:

X = bl_rdata.drop(target_variable, axis=1)
y = bl_rdata[target_variable]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


model = DecisionTreeRegressor(min_samples_split=5, max_depth=5)
model.fit(X_train_scaled, y_train)


y_pred = model.predict(X_test_scaled)


mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Метрики для регрессии (Решающее дерево - Кастомная реализация):")
print(f"  Mean Absolute Error: {mae:.4f}")
print(f"  R-squared: {r2:.4f}")

In [ ]:
  Mean Absolute Error: 0.1592
  R-squared: 0.6377

Метрики не очень сильно отличаются от пункта 2